In [1]:
%matplotlib qt
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def compute_degree(A, B, C):
    BA = A - B
    BC = C - B
    cos_theta = np.dot(BA, BC) / (np.linalg.norm(BA) * np.linalg.norm(BC))
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    angle_rad = np.arccos(cos_theta)
    angle_deg = np.degrees(angle_rad)

    return angle_deg

npy_data = np.load("cropped_porphyrine.npy", allow_pickle=True)

atom_indices = {
    "Fe": 768,
    "N1": 769, "N2": 770, "N3": 771, "N4": 772,
    "S": 773, "H": 774,
    "C1": 775, "C2": 776, "C3": 777, "C4": 779, "C5": 780,
    "C6": 781, "C7": 784, "C8": 785, "C9": 787, "C10": 788,
    "C11": 789, "C12": 792, "C13": 793, "C14": 795, "C15": 796,
    "C16": 797, "C17": 800, "C18": 801, "C19": 803, "C20": 804,
    "O1": 807, "O2": 808
}

new_atom_indices = dict(zip(
                            list(atom_indices.keys()),
                            np.array(list(atom_indices.values())).astype(int) - 768
))


angle_defs = {
    "O2-O1-Fe": ("O2", "O1", "Fe"),
    "O1-Fe-N1": ("O1", "Fe", "N1"),
    "O1-Fe-N2": ("O1", "Fe", "N2"),
    "O1-Fe-N3": ("O1", "Fe", "N3"),
    "O1-Fe-N4": ("O1", "Fe", "N4"),
    "O1-Fe-S":  ("O1", "Fe", "S"),
    "Fe-S-H":   ("Fe", "S", "H"),
    "O1-Fe-C1": ("O1", "Fe", "C1"),
}
carbon_atoms = [k for k in new_atom_indices if k.startswith("C")]
for c_atom in carbon_atoms:
    angle_name = f"O1-Fe-{c_atom}"
    angle_defs[angle_name] = ("O1", "Fe", c_atom)

    angle_name = f"O2-Fe-{c_atom}"
    angle_defs[angle_name] = ("O2", "Fe", c_atom)

    angle_name = f"S-Fe-{c_atom}"
    angle_defs[angle_name] = ("S", "Fe", c_atom)

# Store angles
all_angles = {name: [] for name in angle_defs}

# Process each frame
my_dict = {}
for frame in npy_data:
    coords = {}

    for atom, idx in new_atom_indices.items():
        xyz = frame[idx][1:]
        coords[atom] = xyz

    for angle_name, (a, b, c) in angle_defs.items():
        angle = compute_degree(coords[a], coords[b], coords[c])
        all_angles[angle_name].append(angle)


# Convert to long-format DataFrame
angle_df = pd.DataFrame(all_angles)
angle_df["frame"] = range(len(angle_df))
long_df = angle_df.melt(id_vars="frame", var_name="angle_type", value_name="angle_deg")

In [4]:
## plot hist
df_scsd = pd.read_csv("scsd_50_porphyrine.csv")

g = sns.FacetGrid(data=df_scsd, col="Symm", col_wrap=4)
g.map_dataframe(
                sns.histplot,
                x="Sum (SCSD, Å)",
                bins="auto",
                binwidth=0.2
)
g.set(xticks=np.linspace(0, 5.5, 12), yticks=np.linspace(0, 40, 9))
g.set_xticklabels(np.linspace(0, 5.5, 12), rotation=45)
# g.savefig("/Users/samirabaghbanbari/samira/porphyrin_results/scsd_hist.pdf")

In [30]:
long_df["angle_type"].unique()

array(['O2-O1-Fe', 'O1-Fe-N1', 'O1-Fe-N2', 'O1-Fe-N3', 'O1-Fe-N4',
       'O1-Fe-S', 'Fe-S-H', 'O1-Fe-C1', 'O2-Fe-C1', 'S-Fe-C1', 'O1-Fe-C2',
       'O2-Fe-C2', 'S-Fe-C2', 'O1-Fe-C3', 'O2-Fe-C3', 'S-Fe-C3',
       'O1-Fe-C4', 'O2-Fe-C4', 'S-Fe-C4', 'O1-Fe-C5', 'O2-Fe-C5',
       'S-Fe-C5', 'O1-Fe-C6', 'O2-Fe-C6', 'S-Fe-C6', 'O1-Fe-C7',
       'O2-Fe-C7', 'S-Fe-C7', 'O1-Fe-C8', 'O2-Fe-C8', 'S-Fe-C8',
       'O1-Fe-C9', 'O2-Fe-C9', 'S-Fe-C9', 'O1-Fe-C10', 'O2-Fe-C10',
       'S-Fe-C10', 'O1-Fe-C11', 'O2-Fe-C11', 'S-Fe-C11', 'O1-Fe-C12',
       'O2-Fe-C12', 'S-Fe-C12', 'O1-Fe-C13', 'O2-Fe-C13', 'S-Fe-C13',
       'O1-Fe-C14', 'O2-Fe-C14', 'S-Fe-C14', 'O1-Fe-C15', 'O2-Fe-C15',
       'S-Fe-C15', 'O1-Fe-C16', 'O2-Fe-C16', 'S-Fe-C16', 'O1-Fe-C17',
       'O2-Fe-C17', 'S-Fe-C17', 'O1-Fe-C18', 'O2-Fe-C18', 'S-Fe-C18',
       'O1-Fe-C19', 'O2-Fe-C19', 'S-Fe-C19', 'O1-Fe-C20', 'O2-Fe-C20',
       'S-Fe-C20'], dtype=object)

In [5]:
symm_names = ["B1u", "B2u", "B2g"]
comb_names = ["O2-O1-Fe", "O1-Fe-S", "Fe-S-H"]

dfs_list = []
for item in symm_names:
    df_bu = df_scsd.query(f"Symm == '{item}'")
    sub_dfs_list = []
    for comb in long_df["angle_type"].unique():
        sub_df = long_df.query(f'angle_type == "{comb}"')
        sub_df["Sum (SCSD, Å)"] = df_bu["Sum (SCSD, Å)"].to_list()
        corr = sub_df["angle_deg"].corr(sub_df["Sum (SCSD, Å)"])
        sub_dfs_list.append(sub_df)

    sub_dfs = pd.concat(sub_dfs_list)
    sub_dfs["Symm"] = item
    dfs_list.append(sub_dfs)
    
df_final = pd.concat(dfs_list)

/var/folders/zy/f57pfw6n7jzbdyhtrw5h0jh00000gn/T/ipykernel_2411/1708005061.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["Sum (SCSD, Å)"] = df_bu["Sum (SCSD, Å)"].to_list()
/var/folders/zy/f57pfw6n7jzbdyhtrw5h0jh00000gn/T/ipykernel_2411/1708005061.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["Sum (SCSD, Å)"] = df_bu["Sum (SCSD, Å)"].to_list()
/var/folders/zy/f57pfw6n7jzbdyhtrw5h0jh00000gn/T/ipykernel_2411/1708005061.py:10: SettingWithCopyWarning: 
A value is trying to be set on 

In [7]:
df_final.to_csv("/Users/samirabaghbanbari/samira/porphyrin_results/angles.csv")

In [ ]:
g = sns.FacetGrid(data=df_final, row="Symm", col="angle_type",
                  col_order=comb_names,
                  sharey=True,
                  sharex=True
                  )
g.map_dataframe(
                sns.regplot,
                x="Sum (SCSD, Å)",
                y="angle_deg"
)
# g.savefig("/Users/samirabaghbanbari/samira/porphyrin_results/angles_vs_scsd.pdf")